Connecting drive

In [ ]:
from google.colab import drive  
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Function for reading images**

In [ ]:
import numpy as np
import cv2

def read_images(folder,count):
  data = []
  for i in range(1, count+1):  
    img = cv2.imread('/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+folder+str(i)+'.jpg')
    img_col = np.array(img) 
    #image will be flatten further more...
    subject = int(i)
    data.append(img_col)
  return np.array(data)

In [ ]:
test=read_images('test',50)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if sys.path[0] == '':


**Function to read ground truth and save as jpg image**

In [ ]:
from scipy import io
import os
from imageio import imwrite
import matplotlib.image as mpimg
def read_mat_borderlines(folder,count):
  save_pth1='/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+'border_lines/'
  os.makedirs(save_pth1,exist_ok=True)
  for i in range(1,count+1):
    data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+folder+str(i))
    for j in range (len(data['groundTruth'][0])): 
      edge_data = (data['groundTruth'][0][j][0][0][1]).astype(np.uint8)  
      edge_data_255 = edge_data * 255
      new_img_name = folder.split('.')[0]+'_border_lines'+str(i)+'_'+str(j)+'.jpg'
      imwrite(os.path.join(save_pth1,new_img_name), edge_data_255)     

In [ ]:
def read_mat_colored_segment(folder,count):
  save_pth2='/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+'colored_segments'+'/'
  os.makedirs(save_pth2,exist_ok=True)
  for i in range(1,count+1):
    data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+folder+str(i))
    for j in range (len(data['groundTruth'][0])): 
      edge_data = (data['groundTruth'][0][j][0][0][0]).astype(np.uint8)  
      edge_data_255 = edge_data * 255
      new_img_name = folder.split('.')[0]+'_colored'+str(i)+'_'+str(j)+'.jpg'
      path=os.path.join(save_pth2+new_img_name)
      mpimg.imsave(path,edge_data_255)  

In [ ]:
ground_truth_test_borderlines=read_mat_borderlines('ground_truth_test',50)

In [ ]:
ground_truth_test_colored=read_mat_colored_segment('ground_truth_test',50)

In [ ]:
import random
def k_means(data,k):
  centroids = []
  for i in range(0,k):
    centroids.append(random.choice(data))
  while True:
    cluster_set = np.empty((k, 0)).tolist()  
    newCentroids = []
    cluster_index = []
    distance = []
    for i in range(0,len(data)):
      row = []
      for j in range(0,k):
        row.append(np.linalg.norm(data[i] - centroids[j]))
      distance.append(row)
    distance = np.array(distance)   
    for i in range(0,len(data)):
      for j in range(0,k):
        if (np.argmin(distance[i]))==j:
          cluster_set[j].append(data[i])
          cluster_index.append(j)
    cluster_set= np.array(cluster_set)
    for n in range(k):
      if len(cluster_set[n])==0:
        centroids[n]=random.choice(data)
      else:
        row =sum((cluster_set[n])[:])/len(cluster_set[n])
        newCentroids.append(row)
    newCentroids=centroids.copy()
    if np.sum(newCentroids) != np.sum(centroids):
        centroids= newCentroids
    else:
      break
  cluster_index=np.array(cluster_index)
  return cluster_index

In [ ]:
def save_images(folder,pixels,k,j,number,name):
  save_pth='/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+name+str(number)+'/'
  os.makedirs(save_pth,exist_ok=True)
  new_img_name =str(j)+'_kmeans_'+'_'+str(k)+'.jpg'
  path=os.path.join(save_pth+new_img_name)
  mpimg.imsave(path,pixels)  

In [ ]:
import matplotlib.pyplot as plt
for k in range (1,6):
  for i in range(0,50):
    for j in range (3,12,2):
      pixels=k_means(test[i].reshape(test[i].shape[0]*test[i].shape[1],test[i].shape[2]),j)
      pixels = pixels.reshape(test[i].shape[0],test[i].shape[1])
      save_images('ground_truth_test',pixels,j,i,k,'images')

In [ ]:
def readAndResize(folder,count,scale_percent):
  for i in range(1,count+1):
    img = cv2.imread('/content/gdrive/MyDrive/assigm2_data/'+folder+'/'+folder+str(i)+'.jpg') #need to be edited 
    width = int(img.shape[1] * scale_percent / 100) 
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_image = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    save_pth='/content/gdrive/MyDrive/assigm2_data/'+'resized'+'/'+'res'+str(i)+'.jpg'
    cv2.imwrite(save_pth,resized_image)


In [ ]:
readAndResize('test',50,25)

In [ ]:
def calcDistanceMatrix(data):
  data=data.reshape(data.shape[0]*data.shape[1],data.shape[2])
  distanceMat=[]
  for i in range(data.shape[0]):
    distanceMat.append(np.linalg.norm((data[i]-data),axis=1))
  return np.array(distanceMat)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

def normCut(data, k, knn):

  distMatrix=calcDistanceMatrix(data)
  #connectivity here referes to ones and zeros not distance ,include self refers to not to consider the point itself to be neighbor
  adj = kneighbors_graph(distMatrix , knn , mode='connectivity', include_self=False).toarray() 
  degree = np.diag(np.sum(adj, axis=1))
  L = np.subtract(degree,adj)
  eigen_values, eigen_vectors = np.linalg.eig(np.dot(np.linalg.inv(degree),L))

  sorted_eigen_vectors = eigen_vectors[:,eigen_values.argsort()][:,0:k]
  for i in range(k):
    sorted_eigen_vectors[:,i] = sorted_eigen_vectors[:,i] / np.linalg.norm(sorted_eigen_vectors[:,i])

  km = KMeans(n_clusters=k).fit(np.real(sorted_eigen_vectors))
  return km.labels_

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
def norm_cut_save(num):
  test = cv2.imread('/content/gdrive/MyDrive/assigm2_data/resized/res'+str(num)+'.jpg') #need to be edited 
  result=normCut(test,5,5)
  result = result.reshape(test.shape[0],test.shape[1])
  mpimg.imsave('/content/gdrive/MyDrive/assigm2_data/normalized_cut/'+str(num)+'_Normcut.jpg',result)  

In [ ]:
norm_cut_save(1)
norm_cut_save(2)
norm_cut_save(3)
norm_cut_save(4)
norm_cut_save(20)
norm_cut_save(48)

In [ ]:
def mask(kmeans,ground_truth):
  x,y=np.unique(kmeans,return_counts=True)
  kmeans_unique=sort_lists(x, y)
  x,y=np.unique(ground_truth,return_counts=True)
  ground_truth_unique=sort_lists(x, y)
  if len(kmeans_unique)<=len(ground_truth_unique):
    for i in range(0,len(kmeans_unique)-1):
      ground_truth=np.where(ground_truth==ground_truth_unique[i],kmeans_unique[i]+500,ground_truth)
    ground_truth=np.where(ground_truth<500,kmeans_unique[-1]+500,ground_truth)
    ground_truth=[x - 500 for x in ground_truth]
  else:
    for i in range(0,len(ground_truth_unique)-1):
      kmeans=np.where(kmeans==kmeans_unique[i],ground_truth_unique[i]+500,kmeans)
    kmeans=np.where(kmeans<500,ground_truth_unique[-1]+500,kmeans)
    kmeans=[x - 500 for x in kmeans]
  return kmeans,ground_truth

In [ ]:
def sort_lists(list1, list2):
    zipped_pairs = zip(list2, list1)
    z = [x for _, x in sorted(zipped_pairs,reverse=True)]
    return z

In [ ]:
from sklearn.metrics.cluster import contingency_matrix
import math
def measures():
  f_measure_list=[]
  Conditional_Entropy_list=[]
  for i in range (1,51):
    data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/'+'ground_truth_test'+'/'+'ground_truth_test'+str(i))
    #print('------------------------------')
    #print('Image Number ',i)
    #print('---------------------')
    for k in range(3,12,2):
      f_measure=0
      f_score=0
      Conditional_Entropy=0
      Conditional_Entropy_score=0
      for j in range(0,len(data['groundTruth'][0])):
        ground_truth = (data['groundTruth'][0][j][0][0][0]).astype(np.uint8)
        test = cv2.imread('/content/gdrive/MyDrive/assigm2_data/test/test'+str(i)+'.jpg')  
        pixels=k_means(test.reshape(test.shape[0]*test.shape[1],test.shape[2]),k)
        kmeans_labels,groundTruth_labels=mask(pixels,ground_truth)
        contingency_mat = contingency_matrix (groundTruth_labels,kmeans_labels)
        for x in range (len(contingency_mat[0,:])):
          max = np.argmax(contingency_mat[:,x])
          precision =contingency_mat[max,x]/np.sum(contingency_mat[:,x])
          recall = contingency_mat[max,x]/np.sum(contingency_mat[max,:])
          f_measure+=(2*precision*recall)/(precision+recall)
        for y in range (len(contingency_mat[0,:])):
          entropy = 0
          Conditional_Entropy=0
          for z in range (len(contingency_mat)):
            if(contingency_mat[z][y]!=0.0):
              temp = contingency_mat[z][y]/sum(contingency_mat[:,y])
              entropy-=(temp)*math.log2(temp)
          Conditional_Entropy+=(sum(contingency_mat[:,y])/(len(pixels)))*entropy
          Conditional_Entropy_score+=Conditional_Entropy
        f_measure=f_measure/len(contingency_mat[0,:])
        f_score+=f_measure
      f_score=f_score/len(data['groundTruth'][0]) 
      f_measure_list.append(f_score)
      Conditional_Entropy_score=Conditional_Entropy_score/len(data['groundTruth'][0]) 
      Conditional_Entropy_list.append(Conditional_Entropy_score)
      #print('f_measure (',k,'means) = ',f_score)
      #print('Conditional_Entropy (',k,'means) = ',Conditional_Entropy_score)
  return f_measure_list,Conditional_Entropy_list

In [ ]:
from prettytable import PrettyTable
def print_results(f_measure,conditional_entropy,i):
  print('Image :',i+1) 
  # Specify the Column Names while initializing the Table
  myTable = PrettyTable(["k_means", "F measure", "Conditional entropy"])
 
  # Add rows
  myTable.add_row(["3", f_measure[(i*5)], conditional_entropy[(i*5)]])
  myTable.add_row(["5", f_measure[(i*5)+1], conditional_entropy[(i*5)+1]])
  myTable.add_row(["7", f_measure[(i*5)+2], conditional_entropy[(i*5)+2]])
  myTable.add_row(["9", f_measure[(i*5)+3], conditional_entropy[(i*5)+3]])
  myTable.add_row(["11", f_measure[(i*5)+4], conditional_entropy[(i*5)+4]])

  print(myTable)

In [ ]:
f_measure1,conditional_entropy1=measures()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
for i in range (0,int(len(f_measure1)/5)):
  print_results(f_measure1,conditional_entropy1,i)

Image : 1
+---------+---------------------+---------------------+
| k_means |      F measure      | Conditional entropy |
+---------+---------------------+---------------------+
|    3    |  0.5754384809868774 |  1.300240078294118  |
|    5    | 0.42764476846281163 |  1.7911298914538005 |
|    7    | 0.39831946530711865 |  1.826494292168676  |
|    9    | 0.33460854304829074 |  1.900435125392765  |
|    11   | 0.32368553076403483 |  1.8772767758791613 |
+---------+---------------------+---------------------+
Image : 2
+---------+--------------------+---------------------+
| k_means |     F measure      | Conditional entropy |
+---------+--------------------+---------------------+
|    3    | 0.7118976391482597 |  0.6610827735962425 |
|    5    | 0.6158729341420218 |  0.7264442222773774 |
|    7    | 0.6004715424152759 |  0.7274175959620076 |
|    9    | 0.6227956311254709 |  0.652890263553881  |
|    11   | 0.5303982164575859 |  0.6968891125965325 |
+---------+--------------------+----

In [ ]:
from matplotlib import pyplot as plt
import cv2
def view_edges(image_num):
  fig = plt.figure(figsize=(10,10))
  data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/ground_truth_test'+str(image_num))
  rows = 1
  columns = 1+len(data['groundTruth'][0])
  original = cv2.imread('/content/gdrive/MyDrive/assigm2_data/test/test'+str(image_num)+'.jpg')
  fig.add_subplot(rows, columns,1)
  plt.imshow(original)
  plt.axis('off')
  for j in range(len(data['groundTruth'][0])):
    edge = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/border_lines/ground_truth_test_border_lines'+str(image_num)+'_'+str(j)+'.jpg')
    fig.add_subplot(rows, columns,1+j+1)
    plt.imshow(edge)
    plt.axis('off') 

  fig.savefig('/content/gdrive/MyDrive/assigm2_data/Figures/Original&edges/fig'+str(image_num)+'.jpg',bbox_inches='tight')
    

In [ ]:
for i in range(50):
  view_edges(i+1)

In [ ]:
def view_kmeans(image_num):
  fig = plt.figure(figsize=(12,3))
  data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/ground_truth_test'+str(image_num))
  rows = 1
  columns = 5
  k=3
  for i in range(1,6):
    kmeans = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/images1/'+str(image_num)+'_kmeans__'+str(k)+'.jpg')
    fig.add_subplot(rows, columns,i)
    plt.imshow(kmeans)
    plt.title('Kmeans, k= '+str(k))
    plt.axis('off')
    k=k+2 
  fig.savefig('/content/gdrive/MyDrive/assigm2_data/Figures/Kmeans/fig'+str(image_num)+'.jpg',bbox_inches='tight')

In [ ]:
for i in range (1,51):
  view_kmeans(i)

In [ ]:
def view_kmeansAndGround(image_num):
  fig = plt.figure(figsize=(12,3))
  data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/ground_truth_test'+str(image_num))
  rows = 2
  columns = 7
  k=3
  for i in range(1,6):
    kmeans = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/images1/'+str(image_num)+'_kmeans__'+str(k)+'.jpg')
    fig.add_subplot(rows, columns,i)
    plt.imshow(kmeans)
    plt.title('Kmeans, k= '+str(k))
    plt.axis('off')
    k=k+2 
  for j in range(len(data['groundTruth'][0])):
    edge = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/colored_segments/ground_truth_test_colored'+str(image_num)+'_'+str(j)+'.jpg')
    fig.add_subplot(rows, columns,1+j+1+5+1)
    plt.imshow(edge)
    plt.axis('off')
    plt.title('GT('+str(j+1)+')')

  fig.savefig('/content/gdrive/MyDrive/assigm2_data/Figures/Kmeans&Ground/fig'+str(image_num)+'.jpg',bbox_inches='tight')

In [ ]:
view_kmeansAndGround(2)
view_kmeansAndGround(3)
view_kmeansAndGround(4)
view_kmeansAndGround(20)
view_kmeansAndGround(48)

In [ ]:
def view_groundAndNorm(image_num):
  fig = plt.figure(figsize=(10,10))
  data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/ground_truth_test'+str(image_num))
  rows = 1
  columns = 1+len(data['groundTruth'][0])
  normcut = cv2.imread('/content/gdrive/MyDrive/assigm2_data/normalized_cut/'+str(image_num)+'_Normcut.jpg')
  fig.add_subplot(rows, columns,1)
  plt.imshow(normcut)
  plt.axis('off')
  plt.title("Norm.cut")
  for j in range(len(data['groundTruth'][0])):
    edge = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/colored_segments/ground_truth_test_colored'+str(image_num)+'_'+str(j)+'.jpg')
    fig.add_subplot(rows, columns,1+j+1)
    plt.imshow(edge)
    plt.axis('off')
    plt.title('GT('+str(j+1)+')')

  fig.savefig('/content/gdrive/MyDrive/assigm2_data/Figures/Normalized&Ground/fig'+str(image_num)+'.jpg',bbox_inches='tight')


In [ ]:
view_groundAndNorm(2)
view_groundAndNorm(3)
view_groundAndNorm(4)
view_groundAndNorm(20)
view_groundAndNorm(48)

In [ ]:
def viewkmeansAndNormalized(num):
  fig = plt.figure(figsize=(15,15))
  rows = 1
  columns = 6
  normalized_cut = cv2.imread('/content/gdrive/MyDrive/assigm2_data/normalized_cut/'+str(num)+'_Normcut.jpg')
  fig.add_subplot(rows, columns,1)  
  plt.imshow(normalized_cut)
  plt.axis('off')
  plt.title('Normalized cut')
  k=3
  for j in range(2,7):
    kmeans = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/images1/'+str(num)+'_kmeans__'+str(k)+'.jpg')
    fig.add_subplot(rows, columns,j)
    plt.imshow(kmeans)
    plt.title('Kmeans, k= '+str(k))
    plt.axis('off')
    k=k+2
  
  fig.savefig('/content/gdrive/MyDrive/assigm2_data/Figures/Normalized&Kmeans/fig'+str(num)+'.jpg',bbox_inches='tight')

In [ ]:
viewkmeansAndNormalized(2)
viewkmeansAndNormalized(3)
viewkmeansAndNormalized(4)
viewkmeansAndNormalized(20)
viewkmeansAndNormalized(48)

In [ ]:
import cv2
def kmeans_xy(image_num):
  image = cv2.imread('/content/gdrive/MyDrive/assigm2_data/test/test'+str(image_num)+'.jpg')
  image=np.array(image)
  rows=image.shape[0]
  col=image.shape[1]
  x=[]
  y=[]
  for i in range (rows):
    for j in range(col):
      x.append(i)
      y.append(j)
  x= np.array(x).reshape(rows*col,1)
  y=np.array(y).reshape(rows*col,1)
  image = image.reshape(rows*col,3)
  image = np.append(image,x,axis = 1)
  image = np.append(image,y,axis = 1)
  image=image.reshape(rows*col,5)
  for k in range(3,12,2):
    result=k_means(image,k)
    result = result.reshape(rows,col)
    mpimg.imsave('/content/gdrive/MyDrive/assigm2_data/kmeans_xy/img'+str(image_num)+'_k='+str(k)+'.jpg',result)  

In [ ]:
import numpy as np
kmeans_xy(2)
kmeans_xy(3)
kmeans_xy(4)
kmeans_xy(20)
kmeans_xy(48)

In [ ]:
def view_kmeans_xyAndGround(image_num):
  fig = plt.figure(figsize=(12,3))
  data = io.loadmat('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/ground_truth_test'+str(image_num))
  rows = 2
  columns = 7
  k=3
  for i in range(1,6):
    kmeans = cv2.imread('/content/gdrive/MyDrive/assigm2_data/kmeans_xy/img'+str(image_num)+'_k='+str(k)+'.jpg')
    fig.add_subplot(rows, columns,i)
    plt.imshow(kmeans)
    plt.title('Kmeans, k= '+str(k))
    plt.axis('off')
    k=k+2 
  for j in range(len(data['groundTruth'][0])):
    edge = cv2.imread('/content/gdrive/MyDrive/assigm2_data/ground_truth_test/colored_segments/ground_truth_test_colored'+str(image_num)+'_'+str(j)+'.jpg')
    fig.add_subplot(rows, columns,1+j+1+5+1)
    plt.imshow(edge)
    plt.axis('off')
    plt.title('GT('+str(j+1)+')')

  fig.savefig('/content/gdrive/MyDrive/assigm2_data/Figures/Kmeans_xy&Ground/fig'+str(image_num)+'.jpg',bbox_inches='tight')

In [ ]:
view_kmeans_xyAndGround(2)
view_kmeans_xyAndGround(3)
view_kmeans_xyAndGround(4)
view_kmeans_xyAndGround(20)
view_kmeans_xyAndGround(48)